In [1]:
import matplotlib.pyplot as plt
import numpy as np

%run functions/preprocess.py
%run functions/linear_regression.py

np.set_printoptions(formatter={'float_kind': '{:f}'.format})
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('seaborn')


## Loading the data

In [2]:
path = open('path.txt', 'r').read().strip()
df_data = pd.read_pickle(path)
df_data = df_data.drop(['time of day', 'weekday', 'Unix', 'next_event', 'prev_event',
                        'nextUnix', 'prevUnix', 'prevTime', 'event org:resource'], axis=1)


# should actually be done after splitting
df_data = encode(df_data)
df_data = time_diff(df_data)


In [3]:
def sliding_window(window_size, df):
    """transforms df_data into supervised form 
    with rolling window implementations

    Args:
        window_size (int): size of rolling window

    Returns:
        (X, Y): tuple of input and output arrays
    """

    windows = list(df.rolling(window=window_size))
    for i in windows[window_size-1:]:
        # split into X and Y
        i = i.drop('case concept:name', axis=1)
        temp = i.to_numpy()
        temp = [item for sublist in temp for item in sublist]
        Y.append(temp.pop(-1))
        X.append(temp)
    return None


In [4]:
from sklearn.model_selection import train_test_split
np.random.seed(42)
# randomly sample traces from log
df_data = df_data[df_data['case concept:name'].isin(np.random.choice(
    df_data['case concept:name'].unique(), size=2000, replace=False))]

df_data, df_test = train_test_split(df_data, test_size=0.2, shuffle=False)


In [5]:
X = []
Y = []

# transform into windows
df_data.groupby('case concept:name').apply(lambda x: sliding_window(4, x))
# cross validation and training
cv = cross_validate(np.array(X), np.array(Y))
model = cv[1]


---

## Testing

In [ ]:
# df_test = encode(df_test)
# df_test = time_diff(df_test)


In [ ]:
X = []
Y = []
# transform into windows
df_test.groupby('case concept:name').apply(lambda x: sliding_window(4, x))
y_pred = model.predict(X)

error = cv[0]


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

mse = np.mean([mean_squared_error(i[0], i[1]) for i in error])
r2 = np.mean([r2_score(i[0], i[1]) for i in error])
mae = np.mean([mean_absolute_error(i[0], i[1]) for i in error])
rmse = np.mean([mean_squared_error(i[0], i[1], squared=False) for i in error])
print(f'mse: {np.exp(mse)}\nr^2: {r2}\nrmse: {np.exp(rmse)}\nmae:{np.exp(mae)}')
# print(f'mse: {mse}\nr^2: {r2}\nrmse: {rmse}\nmae:{mae}')


mse: inf
r^2: -3.884551415953214e+16
rmse: inf
mae:inf


/tmp/ipykernel_39145/604088035.py:9: RuntimeWarning: overflow encountered in exp
  print(f'mse: {np.exp(mse)}\nr^2: {r2}\nrmse: {np.exp(rmse)}\nmae:{np.exp(mae)}')


In [ ]:
# cols = df_data.columns.tolist()
# cols.remove('time_diff')
# cols = cols + ['time_diff']
# df_data = df_data[cols]

# categorial = ['case LoanGoal']
# numerical = ['case RequestedAmount']
# df_data.drop(categorial, axis=1).drop(numerical, axis=1)
